In [14]:
import psycopg2
from pandas import read_sql
import configparser

In [15]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

['dwh.cfg']

In [16]:
def create_conn(*args,**kwargs):
    config = kwargs['config']
    try:
        con=psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
        print(con)
        return con
    except Exception as err:
        print(err)

In [17]:
con = create_conn(config=config)

<connection object at 0x7f5f9b7f6d58; dsn: 'user=dwhuser password=xxx dbname=dwh host=dwhcluster.czxt4zpjxnmb.us-west-2.redshift.amazonaws.com port=5439', closed: 0>


In [29]:
# verify the staging event tables count
df_events = read_sql("select count(1) from Sparkify.stg_events", con=con)
df_events.head()

,count
0,8056


In [30]:
# verify the stging songs tables count
df_songs = read_sql("select count(1) from Sparkify.stg_songs", con=con)
df_songs.head()

,count
0,14896


In [31]:
# verify the dimesnion songs table count
df_dimsongs = read_sql("select count(1) from Sparkify.songs", con=con)
df_dimsongs.head()

,count
0,14896


In [32]:
# verify the dimesnion artists table count
df_dimartists = read_sql("select count(1) from Sparkify.artists", con=con)
df_dimartists.head()

,count
0,10025


In [33]:
# verify the dimesnion users table count
df_dimusers = read_sql("select count(1) from Sparkify.users", con=con)
df_dimusers.head()

,count
0,105


In [34]:
# verify the dimesnion time table count
df_dimtime = read_sql("select count(1) from Sparkify.time", con=con)
df_dimtime.head()

,count
0,8023


In [35]:
# verify the Fact table: songplays count
df_fact = read_sql("select count(1) from Sparkify.songplays", con=con)
df_fact.head()

,count
0,819


In [57]:
# Currently which users are playing which songs with songs name,highest duration, artist name and from which location
df = read_sql("Select distinct  \
 us.u_first_name  || ' ' || us.u_last_name  as UserName \
,sng.s_title as SongsName  \
,s_year as Year \
,round(sng.s_duration,2) as Duration \
,art.a_name as ArtistName \
,sp.sp_location as Location \
FROM Sparkify.songs sng  \
inner join Sparkify.artists art  \
on sng.s_artist_id=art.a_artist_id  \
inner join Sparkify.songplays sp  \
on sng.s_song_id =sp.sp_song_id \
AND art.a_artist_id=sp.sp_artist_id \
INNER JOIN Sparkify.users us  \
on us.u_user_id =sp.sp_user_id \
order by Duration desc;", con=con)
df.head()

,username,songsname,year,duration,artistname,location
0,Ryann Smith,Tha,1992,547.13,Aphex Twin,"Palestine, TX"
1,Ryann Smith,Tha,1992,547.13,Polygon Window,"Palestine, TX"
2,Aleena Kirby,Paradigm Shift,1998,535.35,Liquid Tension Experiment,"Waterloo-Cedar Falls, IA"
3,Tegan Levine,I CAN'T GET STARTED,0,497.14,Ron Carter,"Portland-South Portland, ME"
4,Chloe Cuevas,I CAN'T GET STARTED,0,497.14,Ron Carter,"San Francisco-Oakland-Hayward, CA"


In [52]:
# How many users are connected from different parts of world or location
df = read_sql("Select sp.sp_location, \
count(distinct us.u_user_id) UserCount  \
FROM Sparkify.songplays sp  \
INNER JOIN Sparkify.users us  \
on us.u_user_id =sp.sp_user_id  \
group by sp.sp_location  \
order by UserCount desc;", con=con)
df.head()

,sp_location,usercount
0,"New York-Newark-Jersey City, NY-NJ-PA",4
1,"Houston-The Woodlands-Sugar Land, TX",3
2,"Atlanta-Sandy Springs-Roswell, GA",3
3,"San Antonio-New Braunfels, TX",3
4,"Tampa-St. Petersburg-Clearwater, FL",2


In [51]:
# Which is the highest playing songs
df = read_sql(" Select sp.sp_song_id ,  \
count(distinct sp.sp_song_id) as SongsCount  \
FROM Sparkify.songplays sp   \
group by sp.sp_song_id  \
order by SongsCount desc;", con=con)
df.head()

,sp_song_id,songscount
0,SOBBZPM12AB017DF4B,1
1,SOSELMV12A6D4FCF5A,1
2,SODXKJL12A58A7E6E9,1
3,SOFVOQL12A6D4F7456,1
4,SOPANEB12A8C13E81E,1


In [50]:
# Which is the users listening more songs
df= read_sql("  Select us.u_first_name || ' ' || us.u_last_name as UserName  ,count(distinct sp.sp_song_id) as SongsCount \
 FROM Sparkify.songplays sp  \
 INNER JOIN Sparkify.users us \
 on us.u_user_id=sp.sp_user_id  \
 group by UserName  order by SongsCount desc;", con=con)
df.head()

,username,songscount
0,Tegan Levine,97
1,Chloe Cuevas,80
2,Kate Harrell,59
3,Aleena Kirby,49
4,Jacob Klein,36


In [55]:
# How many Male and Female users across location
df= read_sql("Select    \
 sp.sp_location as Location   \
,sum(case when us.u_gender='M' then 1 else 0 end) as MaleCount  \
,sum(case when us.u_gender='F' then 1 else 0 end) as FemaleCount  \
FROM Sparkify.songplays sp    \
INNER JOIN Sparkify.users us    \
on us.u_user_id =sp.sp_user_id  \
group by sp.sp_location   \
order by sp.sp_location ; ", con=con)
df.head()

,location,malecount,femalecount
0,"Atlanta-Sandy Springs-Roswell, GA",0,77
1,"Augusta-Richmond County, GA-SC",0,10
2,"Birmingham-Hoover, AL",54,0
3,"Cedar Rapids, IA",0,2
4,"Chicago-Naperville-Elgin, IL-IN-WI",0,78
